# Ebonite Tutorial

Ebonite is a framework for managing machine learning models and their lifecycle.
One the main features is building services from ML models. Also Ebonite can reliably persist them to database of your choice.

## Installing requirements

In [1]:
! pip install -U ebonite flask flasgger scikit-learn

Requirement already up-to-date: ebonite in /Users/mike0sv/miniconda3/envs/mlmlc2/lib/python3.6/site-packages (0.3.3)
Requirement already up-to-date: flask in /Users/mike0sv/miniconda3/envs/mlmlc2/lib/python3.6/site-packages (1.1.1)
Requirement already up-to-date: flasgger in /Users/mike0sv/miniconda3/envs/mlmlc2/lib/python3.6/site-packages (0.9.3)
Requirement already up-to-date: pandas in /Users/mike0sv/miniconda3/envs/mlmlc2/lib/python3.6/site-packages (0.25.3)
Requirement already up-to-date: scikit-learn in /Users/mike0sv/miniconda3/envs/mlmlc2/lib/python3.6/site-packages (0.22.1)


## Train a model

This is the part where you train your model like you usually do. It can be any type of model from supported framework ([list of supported frameworks](https://github.com/zyfra/ebonite#supported-libraries-and-repositories)). 
If your framework is not supported, you can use any python function as a model or you can easily [implement](https://ebonite.readthedocs.io/en/latest/usage/04_adding_custom_analyzers.html) suppport for your framework.

In this example we will train simple linear regression model from sklearn library.

In [2]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

X, y = load_diabetes(True)

lr = LinearRegression()
lr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now we have `lr` object with trained model

## Create ebonite Model from model object

In [3]:
import ebonite

Let's use `create_model` function to turn our `lr` object into ebonite `Model` object

In [4]:
model = ebonite.create_model(lr, X, model_name='diabetes_model_1')
model

Model(id=None,name=diabetes_model_1)

As you noticed, we also provide sample data when creating `Model`.
This is needed for ebonite to determine input and output data types that this model consumes and produces.
Using this information, ebonite will automatically provide valid interfaces and data serializers.

Now let's see what metadata ebonite got from `lr` object.

In [5]:
from pprint import pprint
from pyjackson import serialize

pprint(serialize(model))

{'author': 'mike0sv',
 'creation_date': '2020-01-18 22:51:50.296290 ',
 'name': 'diabetes_model_1',
 'requirements': {'requirements': [{'module': 'numpy',
                                    'type': 'installable',
                                    'version': '1.17.3'},
                                   {'module': 'sklearn',
                                    'type': 'installable',
                                    'version': '0.22.1'}]},
 'wrapper': {'type': 'ebonite.ext.sklearn.model.SklearnModelWrapper'}}


We can see that ebonite determined type of model (sklearn model) and it's requirements: sklearn for model and numpy for data.`

## Use ebonite client to push Model to repository

Now we can save `Model` to repository. For this example we will use local repository, which stores artifacts and metadata in `.ebonite` directory on local filesystem.
For production, instead you can use different combinations of repositories for metadata and artifacts, for example PostgeSQL db for metadata and S3 bucket for artifacts.

In [6]:
ebnt = ebonite.Ebonite.local(clear=True)
task = ebnt.get_or_create_task('my_prj', 'diabetes_task')
task.push_model(model)

Model(id=68a78d5a-3849-45ef-a728-5da4ac37ec34,name=diabetes_model_1)

We create `Task` object with name `diabetes_task` to store our model, which is stored in `Project` object with name `my_prj`.
Projects and Tasks are needed to add structure to your repository, so you can use one ebonite instance for all different problems you are working on.

In [7]:
model.id

'68a78d5a-3849-45ef-a728-5da4ac37ec34'

Now that we pushed our model, it has an `id` attribute. It can be used to load this model from repository, however model name also can be used for this.

In [8]:
model = ebnt.get_model('diabetes_model_1', task)

In [9]:
model

Model(id=68a78d5a-3849-45ef-a728-5da4ac37ec34,name=diabetes_model_1)

In [10]:
pprint(serialize(model))

{'artifact': {'blobs': {'methods.json': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/68a78d5a-3849-45ef-a728-5da4ac37ec34/methods.json',
                                         'type': 'local_file'},
                        'model.pkl': {'path': '/Users/mike0sv/PycharmProjects/zyfra/ebonite/ebonite/examples/notebook_tutorial/.ebonite/artifacts/68a78d5a-3849-45ef-a728-5da4ac37ec34/model.pkl',
                                      'type': 'local_file'}},
              'type': 'blobs'},
 'author': 'mike0sv',
 'creation_date': '2020-01-18 22:51:50.296290 ',
 'id': '68a78d5a-3849-45ef-a728-5da4ac37ec34',
 'name': 'diabetes_model_1',
 'requirements': {'requirements': [{'module': 'numpy',
                                    'type': 'installable',
                                    'version': '1.17.3'},
                                   {'module': 'sklearn',
                                    'type': 'installable',
           

When we push `Model` to repository, we are not only saving metadata, but also we saving model binary artifacts (i.e. files that contain actual model dump). 
Those artifacts appear in metadata as artifact attribute of our model. Here we can see two local files: `model.pkl`, which is pickled model and `methods.json`, which is metadata about available methods (`predict` and/or `predict_proba` for example)

## Serving model with Flask Server

We also can use `Model` object to create services. For this example we will use nice builtin flask server. But you can implement any type of server you need for your system.


N.B. Running server in jupyter is bad idea, we do this for demo purposes.
Actually `run_model_server` function is mainly for debugging your server or model, 
in production we encourage you to build docker images with your servers


In [11]:
from ebonite.runtime import run_model_server

In [13]:
from ebonite.ext.flask.server import FlaskServer
run_model_server(model, FlaskServer())

2020-01-19 01:53:41,826 [INFO] ebonite_runtime: Starting Ebonite runtime with loader MockLoader and server FlaskServer ...
2020-01-19 01:53:41,840 [INFO] ebonite_runtime: Running server <ebonite.ext.flask.server.FlaskServer object at 0x11c8f2b38>
 * Serving Flask app "ebonite.ext.flask.server" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Jan/2020 01:53:45] "GET /apidocs/ HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 01:53:45] "GET /flasgger_static/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2020 01:53:45] "GET /flasgger_static/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2020 01:53:45] "GET /flasgger_static/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2020 01:53:45] "GET /flasgger_static/lib/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Jan/2020 01:53:46] "GET /apispec_1.json HTTP/1.1" 200 -
127.0.0.1 - - [19/Jan/2020 01:53:50] "POST /predict HTTP/1.1" 400 -
127.0.0.1 - - [19/Jan/2020 01:54:02] "POST /predict HTTP/1.1" 200 -


Now you can check out cool OpenAPI (ex-swagger) UI [here](http://localhost:9000/apidocs)
You can even send some test requests there.

Before you continue, don't forget to stop server cell with `Ctrl+C`

## Building and running docker container with Model

But the best way to deploy your model is to create a docker image with this model and run it on your infrastructure.
You'll need docker up and running on your machine for this to work

In [14]:
from ebonite.ext.flask.helpers import build_model_flask_docker

Here we use helper method to build production-ready flask server with uwsgi and nginx web server.
For less specific build you can use more common function `ebonite.build.build_model_docker`

This step can take a while, so be patient.

In [15]:
build_model_flask_docker('nb_example_diabetes', model, force_overwrite=True)

2020-01-19 01:55:41,905 [INFO] ebonite: Build successful


Now we can run our docker image right from code.

In [21]:
ebnt.run_service('nb_example_diabetes', {80:80}, detach=True) 

Here is the link to the same [OpenAPI UI](http://localhost:80/apidocs)

## Sending requests to service

Builtin flask server also provides an [endpoint](http://localhost:80/interface.json) with it's interface, and we can create a client from it.

In [22]:
from ebonite.ext.flask.client import HTTPClient

In [23]:
client = HTTPClient('localhost', 80)

Now we can send requests to our service using the same data types the unrelying model needs. The client will handle serialization for us.

In [28]:
client.methods['predict']

Method(name='predict', args=[Argument(name='vector', type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None, 10),dtype=float64]'>)], out_type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None,),dtype=float64]'>)

In [25]:
import numpy as np

client.predict(np.array([[0., 1., 2., 3., 4., 5., 6., .7, .8, .9]]))

array([2532.25644396])

## Python function example

Sometimes you need to do some pre- or postprocessing on data. In this case you can create a python function with your logic and use it as a model.
Or you even have a plain python function with some ifs which IS your model.

In [34]:
def is_bad(data):
    preds = lr.predict(data)
    return preds > 150

Lets repeat the same steps for this function.

In [38]:
model2 = ebonite.create_model(is_bad, X, model_name='diabetes_model_2')

In [39]:
pprint(serialize(model2))

{'author': 'mike0sv',
 'creation_date': '2020-01-18 23:14:16.051823 ',
 'name': 'diabetes_model_2',
 'requirements': {'requirements': [{'module': 'numpy',
                                    'type': 'installable',
                                    'version': '1.17.3'},
                                   {'module': 'sklearn',
                                    'type': 'installable',
                                    'version': '0.22.1'}]},
 'wrapper': {'type': 'callable_method'}}


Ebonite still got all the requirements right. 
Let's create and run a service. Note that you actually don't need to save model to do it.

In [42]:
build_model_flask_docker('nb_example_diabetes2', model2, force_overwrite=True)

2020-01-19 02:15:29,267 [INFO] ebonite: Build successful


In [44]:
ebnt.run_service('nb_example_diabetes2', {80:81}, detach=True)

And do some requests.

In [46]:
client2 = HTTPClient('localhost', 81)

In [47]:
client2.methods['predict']

Method(name='predict', args=[Argument(name='vector', type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None, 10),dtype=float64]'>)], out_type=<class 'pyjackson.generics.NumpyNdarrayDatasetType[shape=(None,),dtype=bool]'>)

Note that out_type changed to numpy array of type 'bool'

In [48]:
client2.predict(np.array([[0., 1., 2., 3., 4., 5., 6., .7, .8, .9]]))

array([ True])